<a href="https://colab.research.google.com/github/SeryeongLee/-/blob/main/0531_credit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [58]:
!pip install category_encoders

     |████████████████████████████████| 81kB 7.0MB/s 


In [62]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import os
import re

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor

from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV 
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
from scipy.special import erfc
from sklearn.dummy import DummyRegressor
from sklearn.svm import SVR

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split, cross_validate, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, RandomForestRegressor, VotingRegressor

from lightgbm import LGBMRegressor

import xgboost as xgb
from xgboost import XGBClassifier, XGBRegressor

import warnings

from category_encoders.ordinal import OrdinalEncoder

In [142]:
train = pd.read_csv('/content/drive/MyDrive/creditcard/open/train.csv')
test = pd.read_csv('/content/drive/MyDrive/creditcard/open/test.csv')

In [5]:
train_original = train.copy()
test_original = test.copy()

## 기본 변환

#### 1) 0, 1 변환

#### 2) 수 변환

In [143]:
# 음수 -> 양수 변환

train.DAYS_BIRTH = -train.DAYS_BIRTH
test.DAYS_BIRTH = -test.DAYS_BIRTH

train.DAYS_EMPLOYED = -train.DAYS_EMPLOYED
test.DAYS_EMPLOYED = -test.DAYS_EMPLOYED

train.begin_month = -train.begin_month
test.begin_month = -test.begin_month

In [144]:
# 365243 인 값 0으로 바꾸기

train.DAYS_EMPLOYED = train.DAYS_EMPLOYED.replace([-365243],[0])
test.DAYS_EMPLOYED = test.DAYS_EMPLOYED.replace([-365243],[0])

In [145]:
# 360으로 나눠서 나이, 일한 년수 계산

train['EMPLOYED']= train.DAYS_EMPLOYED.apply(lambda x: abs(x/360) )
train['age']= train.DAYS_BIRTH.apply(lambda x: abs(x/360) ).astype(int)
train['begin_month']= train.begin_month.apply(lambda x: abs(x) ).astype(int)

test['EMPLOYED']= test.DAYS_EMPLOYED.apply(lambda x: abs(x/360) )
test['age']= test.DAYS_BIRTH.apply(lambda x: abs(x/360) ).astype(int)
test['begin_month']= test.begin_month.apply(lambda x: abs(x) ).astype(int)

#### 3) 로그변환

In [146]:
tr_it=train['income_total']
tr_it_log = np.log1p(tr_it)
train['income_total']= tr_it_log
te_it=test['income_total']
te_it_log = np.log1p(te_it)
test['income_total']= te_it_log

In [147]:
train['EMPLOYED']= np.log1p(train['EMPLOYED'])
test['EMPLOYED']= np.log1p(test['EMPLOYED'])

#### 4) child_num, family_size 관련 컬럼 추가

In [148]:
train['child_num'].value_counts()

0     18340
1      5386
2      2362
3       306
4        47
5        10
14        3
7         2
19        1
Name: child_num, dtype: int64

In [149]:
# child_num이 14, 19인 행 지우기

idx_child_drop = train[(train['child_num'] == 14)|(train['child_num'] == 19)].index
train = train.drop(idx_child_drop)

In [150]:
# child_num이 family_size보다 큰 행도 지우기

idx_child_drop2 = train[train['family_size'] < train['child_num']].index
train = train.drop(idx_child_drop2)

In [151]:
train['cf_ratio'] = train['child_num'] / train['family_size']
test['cf_ratio'] = test['child_num'] / test['family_size']

# train['if_ratio'] = train['income_total'] / train['family_size']
# test['if_ratio'] = test['income_total'] / test['family_size']

In [152]:
train['family_size'].value_counts()

2.0    14106
1.0     5108
3.0     4632
4.0     2260
5.0      291
6.0       44
7.0        9
9.0        2
Name: family_size, dtype: int64

In [153]:
# family_size > 7 인 것도 지우기
train = train[(train['family_size']<=7)]
train = train.reset_index(drop=True)

#### 5) 결측치 채우기 - unemployed 만들고 nan 처리

In [154]:
def unemployed(data):
  data.loc[(data['DAYS_EMPLOYED'] == 0), 'occyp_type'] = 'unemployed'

unemployed(train)
unemployed(test)

In [155]:
train.fillna('Nan', inplace=True)
test.fillna('Nan',inplace=True)


In [156]:
train.head()

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit,EMPLOYED,age,cf_ratio
0,0,F,N,N,0,12.218500,Commercial associate,Higher education,Married,Municipal apartment,13899,4709,1,0,0,0,Nan,2.0,6,1.0,2.644795,38,0.000000
1,1,F,N,Y,1,12.419170,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,11380,1540,1,0,0,1,Laborers,3.0,5,1.0,1.663505,31,0.333333
2,2,M,Y,Y,0,13.017005,Working,Higher education,Married,House / apartment,19087,4434,1,0,1,0,Managers,2.0,22,2.0,2.589016,53,0.000000
3,3,F,N,Y,0,12.218500,Commercial associate,Secondary / secondary special,Married,House / apartment,15088,2092,1,0,1,0,Sales staff,2.0,37,0.0,1.918555,41,0.000000
4,4,F,Y,Y,0,11.967187,State servant,Higher education,Married,House / apartment,15037,2105,1,0,0,0,Managers,2.0,26,2.0,1.923843,41,0.000000


#### 명목형 변수 처리 - OrdinalEncoder

In [157]:
encoder = OrdinalEncoder()

In [158]:
cate_feature = train.dtypes[train.dtypes == 'object'].index.tolist()
print(len(cate_feature))
print(train.dtypes)

8
index              int64
gender            object
car               object
reality           object
child_num          int64
income_total     float64
income_type       object
edu_type          object
family_type       object
house_type        object
DAYS_BIRTH         int64
DAYS_EMPLOYED      int64
FLAG_MOBIL         int64
work_phone         int64
phone              int64
email              int64
occyp_type        object
family_size      float64
begin_month        int64
credit           float64
EMPLOYED         float64
age                int64
cf_ratio         float64
dtype: object


In [159]:
train[cate_feature] = encoder.fit_transform(train[cate_feature],train['credit'])
test[cate_feature] = encoder.fit_transform(test[cate_feature])

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [160]:
train = train.drop(['index', 'FLAG_MOBIL', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'child_num','cf_ratio'], axis = 1)

In [161]:
train.head()

,gender,car,reality,income_total,income_type,edu_type,family_type,house_type,work_phone,phone,email,occyp_type,family_size,begin_month,credit,EMPLOYED,age
0,1,1,1,12.218500,1,1,1,1,0,0,0,1,2.0,6,1.0,2.644795,38
1,1,1,2,12.419170,1,2,2,2,0,0,1,2,3.0,5,1.0,1.663505,31
2,2,2,2,13.017005,2,1,1,2,0,1,0,3,2.0,22,2.0,2.589016,53
3,1,1,2,12.218500,1,2,1,2,0,1,0,4,2.0,37,0.0,1.918555,41
4,1,2,2,11.967187,3,1,1,2,0,0,0,3,2.0,26,2.0,1.923843,41


In [ ]:
# 간단한 autoencoder 모델 구현

AutoEncoder + NN 구현

* 입출력이 같은 구조
* input data를 잘 복원한다.



In [163]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model

인코더 작성

In [173]:
input_dim = Input(shape=(16,)) # 입력 채널 수: 16

In [174]:
encoder1 = Dense(8, activation = 'relu')(input_dim)
encoder2 = Dense(3, activation = 'relu')(encoder1)

In [175]:
decoder1 = Dense(8, activation='relu')(encoder2)
decoder2 = Dense(16, activation='relu')(decoder1)

모델 작성 및 컴파일

In [176]:
autoencoder = Model(inputs = input_dim, outputs = decoder2)


In [177]:
autoencoder.compile(optimizer = 'adam', loss = 'binary_crossentropy')

In [169]:
train_y = train['credit']


In [171]:
train_x = train.drop("credit", axis = 1)


x_train, x_test, y_train, y_test = train_test_split(train_x, train_y,
                 stratify = train_y, 
                 test_size = 0.2,
                 random_state = 10086
                 )


In [178]:
autoencoder.fit(x_train, x_train, batch_size= 32, shuffle= True, validation_data = (x_test,x_test))

662/662 [==============================] - 2s 2ms/step - loss: 65.9169 - val_loss: 60.6431


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class Net(nn.Module):
  def __init__(self): # 초기화
    super(Net,self).__init__()
    self.conv1

명목형 변수 처리 - AutoEncoder  
(https://dacon.io/competitions/official/235713/codeshare/2562?page=3&dtype=recent) 참고